In [ ]:
import pandas as pd

import larch as lx
from larch.util.namespaces import Namespace

In [ ]:
spec_content = """expression,DA,SR,Walk,Bike,Transit
AUTO_TIME,InVehTime,InVehTime,,,
AUTO_COST,Cost,,,,
AUTO_COST*0.5,,Cost,,,
1,,ASC_SR,ASC_Walk,ASC_Bike,ASC_Transit
log(INCOME),,LogIncome_SR,LogIncome_Walk,LogIncome_Bike,LogIncome_Transit
WALK_TIME,,,NonMotorTime,,
BIKE_TIME,,,,NonMotorTime,
TRANSIT_IVTT,,,,,InVehTime
TRANSIT_OVTT,,,,,OutVehTime
TRANSIT_FARE,,,,,Cost
"""

from io import StringIO

spec = pd.read_csv(StringIO(spec_content)).set_index("expression")
spec

In [ ]:
hh, pp, tour, skims = lx.example(200, ["hh", "pp", "tour", "skims"])

In [ ]:
Mode = Namespace(
    DA=1,
    SR=2,
    Walk=3,
    Bike=4,
    Transit=5,
)

In [ ]:
# tour_dataset = lx.Dataset.construct.from_idco(tour.set_index('TOURID'), alts=Mode)

tour_dataset = tour.set_index("TOURID")
od_skims = lx.Dataset.construct.from_omx(skims)
dt = lx.DataTree(
    tour=tour_dataset,
    hh=hh.set_index("HHID"),
    person=pp.set_index("PERSONID"),
    od=od_skims,
    do=od_skims,
    relationships=(
        "tours.HHID @ hh.HHID",
        "tours.PERSONID @ person.PERSONID",
        "hh.HOMETAZ @ od.otaz",
        "tours.DTAZ @ od.dtaz",
        "hh.HOMETAZ @ do.dtaz",
        "tours.DTAZ @ do.otaz",
    ),
    root_node_name="tour",
)

In [ ]:
spec.columns

In [ ]:
dt.set_altnames(spec.columns)

In [ ]:
dt.alts_name_to_id()

In [ ]:
import larch.model.from_spec

In [ ]:
m = lx.Model.from_spec(spec, dt)

In [ ]:
m.choice_co_code = "TOURMODE"

In [ ]:
m.should_preload_data()

In [ ]:
m.maximize_loglike()

In [ ]:
dt.n_cases

In [ ]:
m.logsums()

In [ ]:
m.pvals = {"Cost": -0.1, "InVehTime": -0.2, "ASC_Bike": 2.0}

In [ ]:
m.pf